In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/non-fulfillment/sample_submission.csv
/kaggle/input/non-fulfillment/train.csv
/kaggle/input/non-fulfillment/test.csv


In [2]:
!pip install -q pytorch-tabnet tab-transformer-pytorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 3.2 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings('ignore')

# PyTorch 관련 라이브러리
import torch
import torch.nn as nn
import torch.optim as optim

# TabNet (pytorch-tabnet)
from pytorch_tabnet.tab_model import TabNetClassifier

# TabTransformer (tab-transformer-pytorch)
from tab_transformer_pytorch import TabTransformer

import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
    

# 딥러닝 모델 (TensorFlow)
import tensorflow as tf
from tensorflow import keras


In [5]:
# # 데이터 로드 및 전처리
# train_data = pd.read_csv('/kaggle/input/non-fulfillment/train.csv')
# test_data = pd.read_csv('/kaggle/input/non-fulfillment/test.csv')

# # UID 컬럼 제거
# train_data.drop(columns=['UID'], inplace=True)
# test_data.drop(columns=['UID'], inplace=True)

In [6]:
# 1. 데이터 로드 및 UID 제거
train_data = pd.read_csv('/kaggle/input/non-fulfillment/train.csv')
test_data = pd.read_csv('/kaggle/input/non-fulfillment/test.csv')
train_df = pd.read_csv('/kaggle/input/non-fulfillment/train.csv')
test_df = pd.read_csv('/kaggle/input/non-fulfillment/test.csv')
train_data.drop(columns=['UID'], inplace=True)
test_data.drop(columns=['UID'], inplace=True)


In [7]:
train_data.head()

,주거 형태,연간 소득,현재 직장 근속 연수,체납 세금 압류 횟수,개설된 신용계좌 수,신용 거래 연수,최대 신용한도,신용 문제 발생 횟수,마지막 연체 이후 경과 개월 수,개인 파산 횟수,대출 목적,대출 상환 기간,현재 대출 잔액,현재 미상환 신용액,월 상환 부채액,신용 점수,채무 불이행 여부
0,자가,1941337.5,10년 이상,0.0,9,13.4,400597.5,0,24,1,부채 통합,단기 상환,390903.0,225457.5,8806.5,767,0
1,월세,1979505.0,10년 이상,0.0,5,15.1,360679.5,0,11,0,부채 통합,단기 상환,1002184.5,64749.0,24961.5,767,0
2,월세,1356381.0,4년,0.0,12,18.8,491770.5,1,74,3,부채 통합,단기 상환,227775.0,487644.0,12069.0,800,1
3,월세,1049017.5,6년,0.0,15,14.8,411546.0,1,22,1,부채 통합,단기 상환,251383.5,413211.0,31749.0,796,1
4,월세,4320217.5,2년,0.0,11,26.1,895288.5,0,32,0,부채 통합,장기 상환,1163176.5,78991.5,5862.0,751,0


In [8]:
test_data.head()

,주거 형태,연간 소득,현재 직장 근속 연수,체납 세금 압류 횟수,개설된 신용계좌 수,신용 거래 연수,최대 신용한도,신용 문제 발생 횟수,마지막 연체 이후 경과 개월 수,개인 파산 횟수,대출 목적,대출 상환 기간,현재 대출 잔액,현재 미상환 신용액,월 상환 부채액,신용 점수
0,월세,1560090.0,10년 이상,0.0,13,12.0,495561.0,0,18,0,기타,단기 상환,376332.0,133522.5,29641.5,736
1,주택 담보 대출 (거주 중),2102616.0,2년,0.0,9,29.0,580833.0,0,40,0,부채 통합,장기 상환,830379.0,302983.5,20151.0,718
2,주택 담보 대출 (거주 중),2477989.5,10년 이상,0.0,11,26.5,995841.0,0,44,0,부채 통합,장기 상환,877635.0,379278.0,13113.0,722
3,주택 담보 대출 (거주 중),1571091.0,6년,0.0,7,34.4,601656.0,0,45,0,부채 통합,단기 상환,487278.0,275395.5,11679.0,762
4,주택 담보 대출 (거주 중),2290260.0,10년 이상,0.0,19,25.0,1954623.0,0,14,0,부채 통합,단기 상환,397782.0,742767.0,42370.5,775


In [9]:
print(train_data.shape[0])
print(test_data.shape[0])

10000
2062


In [10]:
# # train_data가 이미 DataFrame 형태로 있다고 가정합니다.
# # 수치형 변수들의 상관관계 계산 (문자형 변수는 자동으로 제외됨)
# corr_matrix = train_data.corr()
# print(corr_matrix)

# # 히트맵 시각화
# plt.figure(figsize=(12, 10))
# sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
# plt.title("Train Data - Correlation Matrix")
# plt.show()

In [11]:
print(train_data.columns)

Index(['주거 형태', '연간 소득', '현재 직장 근속 연수', '체납 세금 압류 횟수', '개설된 신용계좌 수',
       '신용 거래 연수', '최대 신용한도', '신용 문제 발생 횟수', '마지막 연체 이후 경과 개월 수', '개인 파산 횟수',
       '대출 목적', '대출 상환 기간', '현재 대출 잔액', '현재 미상환 신용액', '월 상환 부채액', '신용 점수',
       '채무 불이행 여부'],
      dtype='object')


In [12]:
print(test_df.columns)

Index(['UID', '주거 형태', '연간 소득', '현재 직장 근속 연수', '체납 세금 압류 횟수', '개설된 신용계좌 수',
       '신용 거래 연수', '최대 신용한도', '신용 문제 발생 횟수', '마지막 연체 이후 경과 개월 수', '개인 파산 횟수',
       '대출 목적', '대출 상환 기간', '현재 대출 잔액', '현재 미상환 신용액', '월 상환 부채액', '신용 점수'],
      dtype='object')


In [13]:
# 2. 범주형 변수 처리 (라벨 인코딩 & 원-핫 인코딩)
label_enc = LabelEncoder()
train_data["현재 직장 근속 연수"] = label_enc.fit_transform(train_data["현재 직장 근속 연수"])
test_data["현재 직장 근속 연수"] = label_enc.transform(test_data["현재 직장 근속 연수"])

train_data = pd.get_dummies(train_data, columns=["주거 형태", "대출 목적", "대출 상환 기간"], drop_first=True)
test_data = pd.get_dummies(test_data, columns=["주거 형태", "대출 목적", "대출 상환 기간"], drop_first=True)

# 3. 수치형 변수 변환 (로그 변환 및 "연체 없음" 컬럼 추가)
log_columns = ["현재 미상환 신용액", "월 상환 부채액", "현재 대출 잔액"]
for col in log_columns:
    train_data[col] = np.log1p(train_data[col])
    test_data[col] = np.log1p(test_data[col])

train_data["연체 없음"] = (train_data["마지막 연체 이후 경과 개월 수"] == 0).astype(int)
test_data["연체 없음"] = (test_data["마지막 연체 이후 경과 개월 수"] == 0).astype(int)

# 4. 파생 변수 생성 (분모에 epsilon 추가)
epsilon = 1e-8

train_data['DTI'] = (train_data['월 상환 부채액'] * 12) / (train_data['연간 소득'] + epsilon) * 100
test_data['DTI'] = (test_data['월 상환 부채액'] * 12) / (test_data['연간 소득'] + epsilon) * 100

train_data['DSR'] = (train_data['현재 대출 잔액'] + train_data['현재 미상환 신용액']) / (train_data['연간 소득'] + epsilon) * 100
test_data['DSR'] = (test_data['현재 대출 잔액'] + test_data['현재 미상환 신용액']) / (test_data['연간 소득'] + epsilon) * 100

train_data['LTV'] = train_data['현재 대출 잔액'] / (train_data['최대 신용한도'] + epsilon) * 100
test_data['LTV'] = test_data['현재 대출 잔액'] / (test_data['최대 신용한도'] + epsilon) * 100

# 5. 무한대 값을 np.nan으로 치환
train_data.replace([np.inf, -np.inf], np.nan, inplace=True)
test_data.replace([np.inf, -np.inf], np.nan, inplace=True)

# 6. 결측값 처리 (SimpleImputer: 중간값으로 대체)
target = "채무 불이행 여부"
train_features = train_data.drop(columns=[target])
train_target = train_data[target]

imputer = SimpleImputer(strategy="median")
train_features_imputed = pd.DataFrame(imputer.fit_transform(train_features), 
                                      columns=train_features.columns)
test_data_imputed = pd.DataFrame(imputer.transform(test_data), 
                                 columns=test_data.columns)

# 7. 타깃 및 피처 분리 및 train/validation 분할 (SMOTE, Scaling 제거)
X = train_features_imputed.values
y = train_target.values.astype(np.float32)
X_test = test_data_imputed.values

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [14]:
train_data.head()

,연간 소득,현재 직장 근속 연수,체납 세금 압류 횟수,개설된 신용계좌 수,신용 거래 연수,최대 신용한도,신용 문제 발생 횟수,마지막 연체 이후 경과 개월 수,개인 파산 횟수,현재 대출 잔액,...,대출 목적_이사 비용,대출 목적_자동차 구매,대출 목적_주택 개보수,대출 목적_주택 구매,대출 목적_휴가 비용,대출 상환 기간_장기 상환,연체 없음,DTI,DSR,LTV
0,1941337.5,0,0.0,9,13.4,400597.5,0,24,1,12.876217,...,False,False,False,False,False,False,0,0.005615,0.001298,0.003214
1,1979505.0,0,0.0,5,15.1,360679.5,0,11,0,13.817694,...,False,False,False,False,False,False,0,0.006138,0.001258,0.003831
2,1356381.0,5,0.0,12,18.8,491770.5,1,74,3,12.336118,...,False,False,False,False,False,False,0,0.008315,0.001875,0.002509
3,1049017.5,7,0.0,15,14.8,411546.0,1,22,1,12.434739,...,False,False,False,False,False,False,0,0.011858,0.002418,0.003021
4,4320217.5,3,0.0,11,26.1,895288.5,0,32,0,13.966666,...,False,False,False,False,False,True,0,0.002410,0.000584,0.001560


In [15]:
test_data.head()

,연간 소득,현재 직장 근속 연수,체납 세금 압류 횟수,개설된 신용계좌 수,신용 거래 연수,최대 신용한도,신용 문제 발생 횟수,마지막 연체 이후 경과 개월 수,개인 파산 횟수,현재 대출 잔액,...,대출 목적_이사 비용,대출 목적_자동차 구매,대출 목적_주택 개보수,대출 목적_주택 구매,대출 목적_휴가 비용,대출 상환 기간_장기 상환,연체 없음,DTI,DSR,LTV
0,1560090.0,0,0.0,13,12.0,495561.0,0,18,0,12.838230,...,False,False,False,False,False,False,0,0.007920,0.001579,0.002591
1,2102616.0,3,0.0,9,29.0,580833.0,0,40,0,13.629639,...,False,False,False,False,False,True,0,0.005656,0.001248,0.002347
2,2477989.5,0,0.0,11,26.5,995841.0,0,44,0,13.684987,...,False,False,False,False,False,True,0,0.004592,0.001071,0.001374
3,1571091.0,7,0.0,7,34.4,601656.0,0,45,0,13.096592,...,False,False,False,False,False,False,0,0.007153,0.001631,0.002177
4,2290260.0,0,0.0,19,25.0,1954623.0,0,14,0,12.893662,...,False,False,False,False,False,False,0,0.005582,0.001153,0.000660


In [16]:
# train_data_final.head()

In [17]:
# test_data_final.head()

In [18]:
# 9. TabNet 모델 학습
tabnet_clf = TabNetClassifier(seed=42, verbose=1, optimizer_params=dict(lr=2e-2))
tabnet_clf.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    eval_metric=['auc'],
    max_epochs=100,
    patience=10,
    batch_size=1024,
    virtual_batch_size=128
)
tabnet_val_preds = tabnet_clf.predict_proba(X_val)[:, 1]
tabnet_auc = roc_auc_score(y_val, tabnet_val_preds)
print("TabNet Validation AUC:", tabnet_auc)

epoch 0  | loss: 0.7315  | val_0_auc: 0.49943 |  0:00:01s
epoch 1  | loss: 0.64926 | val_0_auc: 0.5601  |  0:00:01s
epoch 2  | loss: 0.62623 | val_0_auc: 0.52956 |  0:00:01s
epoch 3  | loss: 0.61562 | val_0_auc: 0.54246 |  0:00:02s
epoch 4  | loss: 0.60783 | val_0_auc: 0.5583  |  0:00:02s
epoch 5  | loss: 0.59728 | val_0_auc: 0.48303 |  0:00:02s
epoch 6  | loss: 0.59732 | val_0_auc: 0.50919 |  0:00:03s
epoch 7  | loss: 0.591   | val_0_auc: 0.53655 |  0:00:03s
epoch 8  | loss: 0.58772 | val_0_auc: 0.59631 |  0:00:03s
epoch 9  | loss: 0.58816 | val_0_auc: 0.60632 |  0:00:03s
epoch 10 | loss: 0.58604 | val_0_auc: 0.62836 |  0:00:04s
epoch 11 | loss: 0.57803 | val_0_auc: 0.65489 |  0:00:04s
epoch 12 | loss: 0.57873 | val_0_auc: 0.64849 |  0:00:04s
epoch 13 | loss: 0.57791 | val_0_auc: 0.63909 |  0:00:05s
epoch 14 | loss: 0.57681 | val_0_auc: 0.62494 |  0:00:05s
epoch 15 | loss: 0.57299 | val_0_auc: 0.61011 |  0:00:05s
epoch 16 | loss: 0.57533 | val_0_auc: 0.59304 |  0:00:06s
epoch 17 | los

In [19]:
# 10. TabTransformer 모델 구현 및 학습
class TabTransformerClassifier(nn.Module):
    def __init__(self, num_continuous, num_classes=1, dim=32, depth=6, heads=8, mlp_hidden_mults=(4,), dropout=0.1):
        super().__init__()
        # 범주형 피처가 없으므로 빈 텐서를 전달합니다.
        self.tabtransformer = TabTransformer(
            categories=(), 
            num_continuous=num_continuous, 
            dim=dim,
            depth=depth,
            heads=heads,
            attn_dropout=dropout,
            ff_dropout=dropout,
            mlp_hidden_mults=mlp_hidden_mults
        )
        # 출력 차원 확장을 위한 선형 변환 레이어
        self.proj = nn.Linear(1, dim)
        self.fc = nn.Linear(dim, num_classes)
        
    def forward(self, x):
        x_cat = torch.empty(x.size(0), 0, dtype=torch.long, device=x.device)
        x = self.tabtransformer(x_cat, x)
        x = self.proj(x)
        return self.fc(x)

def train_tabtransformer(model, X_train, y_train, X_val, y_val, epochs=50, batch_size=256, lr=1e-3, device='cuda' if torch.cuda.is_available() else 'cpu'):
    model.to(device)
    criterion = nn.BCEWithLogitsLoss()  # 로짓 출력에 적합한 손실 함수
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    train_dataset = torch.utils.data.TensorDataset(torch.tensor(X_train, dtype=torch.float32),
                                                     torch.tensor(y_train, dtype=torch.float32))
    val_dataset = torch.utils.data.TensorDataset(torch.tensor(X_val, dtype=torch.float32),
                                                   torch.tensor(y_val, dtype=torch.float32))
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    best_val_auc = 0
    best_state = None
    for epoch in range(epochs):
        model.train()
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            outputs = model(xb).squeeze(1)
            loss = criterion(outputs, yb)
            loss.backward()
            optimizer.step()
        
        # 검증 단계
        model.eval()
        all_preds = []
        all_targets = []
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                outputs = model(xb).squeeze(1)
                preds = torch.sigmoid(outputs)
                all_preds.append(preds.cpu().numpy())
                all_targets.append(yb.cpu().numpy())
        all_preds = np.concatenate(all_preds)
        all_targets = np.concatenate(all_targets)
        curr_auc = roc_auc_score(all_targets, all_preds)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}, Val AUC: {curr_auc:.4f}")
        if curr_auc > best_val_auc:
            best_val_auc = curr_auc
            best_state = model.state_dict()
    model.load_state_dict(best_state)
    return model

num_features = X_train.shape[1]
tabtrans_model = TabTransformerClassifier(num_continuous=num_features, dim=32, depth=6, heads=8, mlp_hidden_mults=(4,), dropout=0.1)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tabtrans_model = train_tabtransformer(tabtrans_model, X_train, y_train, X_val, y_val, epochs=50, batch_size=256, lr=1e-3, device=device)

tabtrans_model.eval()
with torch.no_grad():
    X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
    outputs = tabtrans_model(X_val_tensor).squeeze(1)
    tabtrans_val_preds = torch.sigmoid(outputs).cpu().numpy()
tabtrans_auc = roc_auc_score(y_val, tabtrans_val_preds)
print("TabTransformer Validation AUC:", tabtrans_auc)

Epoch 1/50, Loss: 0.6416, Val AUC: 0.5403
Epoch 2/50, Loss: 0.6216, Val AUC: 0.5409
Epoch 3/50, Loss: 0.6485, Val AUC: 0.5446
Epoch 4/50, Loss: 0.5995, Val AUC: 0.5452
Epoch 5/50, Loss: 0.6654, Val AUC: 0.5488
Epoch 6/50, Loss: 0.6024, Val AUC: 0.5501
Epoch 7/50, Loss: 0.5924, Val AUC: 0.5510
Epoch 8/50, Loss: 0.6287, Val AUC: 0.5518
Epoch 9/50, Loss: 0.6995, Val AUC: 0.5529
Epoch 10/50, Loss: 0.5809, Val AUC: 0.5527
Epoch 11/50, Loss: 0.6500, Val AUC: 0.5531
Epoch 12/50, Loss: 0.6825, Val AUC: 0.5531
Epoch 13/50, Loss: 0.5838, Val AUC: 0.5537
Epoch 14/50, Loss: 0.6809, Val AUC: 0.5530
Epoch 15/50, Loss: 0.5401, Val AUC: 0.5540
Epoch 16/50, Loss: 0.6645, Val AUC: 0.5538
Epoch 17/50, Loss: 0.6076, Val AUC: 0.5540
Epoch 18/50, Loss: 0.5909, Val AUC: 0.5539
Epoch 19/50, Loss: 0.6907, Val AUC: 0.5543
Epoch 20/50, Loss: 0.6055, Val AUC: 0.5542
Epoch 21/50, Loss: 0.6793, Val AUC: 0.5544
Epoch 22/50, Loss: 0.7123, Val AUC: 0.5545
Epoch 23/50, Loss: 0.6221, Val AUC: 0.5545
Epoch 24/50, Loss: 0

In [20]:
# 11. 단순 평균 앙상블
# TabNet과 TabTransformer의 예측을 단순 평균으로 결합
tabnet_test_preds = tabnet_clf.predict_proba(X_test)[:, 1]
with torch.no_grad():
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
    outputs = tabtrans_model(X_test_tensor).squeeze(1)
    tabtrans_test_preds = torch.sigmoid(outputs).cpu().numpy()

ensemble_preds = (tabnet_test_preds + tabtrans_test_preds) / 2.0


In [21]:
# 12. 제출 파일 생성
submission = pd.read_csv('/kaggle/input/non-fulfillment/sample_submission.csv')
submission['채무 불이행 확률'] = ensemble_preds
submission.to_csv('submission_stacking.csv', index=False)
print("스태킹 앙상블을 이용한 예측 및 제출 파일 생성 완료!")

스태킹 앙상블을 이용한 예측 및 제출 파일 생성 완료!
